In [2]:
import json
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import SSMContainer
from jesse.helpers import date_to_timestamp

from pathlib import Path
import lightgbm as lgb
import pandas as pd
import numpy as np

from research.plot import analyze_confidence_slices

MODEL_TYPE = "r"
THRESHOLD = 0
LOWER_BOUND = -1
UPPER_BOUND = 1


for l in range(4, 8):
    for pn in range(1, 4):
        LAG = l
        PRED_NEXT = pn
        MODEL_NAME = f"{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}"

        STRATEGY_NAME = "BinanceBtcDemoBarV2"

        T_START = "2025-06-01"
        T_END = "2025-11-25"

        path_features = Path(".").parent / "strategies" / f"{STRATEGY_NAME}" / "models" / "feature_info.json"
        with open(path_features) as f:
            feature_info: dict[str, list[str]] = json.load(f)
        FEAT_FRACDIFF: list[str] = feature_info["fracdiff"]
        ALL_RAW_FEAT = []
        ALL_RAW_FEAT.extend(FEAT_FRACDIFF)
        ALL_RAW_FEAT.extend(feature_info[MODEL_NAME])
        ALL_RAW_FEAT = set(ALL_RAW_FEAT)
        ALL_RAW_FEAT = sorted(
            [
                i
                for i in ALL_RAW_FEAT
                if not i.startswith("deep_ssm") and not i.startswith("lg_ssm")
            ]
        )

        candle_container = FusionCandles(
            exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
        )
        candles = candle_container.get_candles(T_START, T_END)

        feat_calc = SimpleFeatureCalculator()
        feat_calc.load(candles, sequential=True)
        print("getting raw features...")
        df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
        df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
        print(f"{df_raw_feats.shape = }")

        model_deep_ssm = SSMContainer(model_type="deep_ssm")
        model_lg_ssm = SSMContainer(model_type="lg_ssm")

        df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
        df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
        df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

        test_features = df_feat_full[
            (df_feat_full.index.to_numpy() >= date_to_timestamp(T_START)) &
            (df_feat_full.index.to_numpy() < date_to_timestamp(T_END))
            ]

        with open(f"strategies/{STRATEGY_NAME}/models/feature_info.json", "r") as f:
            feat_info = json.load(f)
        df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

        timestamps = candles[:, 0].astype(np.int64)
        mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

        close_price = candles[:, 2][mask]
        datetimes = pd.to_datetime(test_features.index, unit="ms")
        assert len(datetimes) == len(close_price)

        model = lgb.Booster(model_file=f"strategies/{STRATEGY_NAME}/models/model_{MODEL_NAME}.txt")
        pred = model.predict(df_selected_feats)
        assert len(pred) == len(datetimes)

        slice_res = analyze_confidence_slices(
            datetimes,
            pred,
            close_price,
            PRED_NEXT,
            output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
            granularity=0.01,
            lower_bound=LOWER_BOUND,
            upper_bound=UPPER_BOUND,
            threshold=THRESHOLD,
        )

getting raw features...
df_raw_feats.shape = (1400, 3095)
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/deep_ssm.json
Model loaded from /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.safetensors and /Users/yangqiuyu/Github/jesse-trade/strategies/BinanceBtcDeapV1Voting/models/lg_ssm.json

⚠️ 警告：发现 119 个超出范围 [-1, 1] 的值
  - 占比: 8.50%
  - 实际范围: [-1.1915, 1.1118]
  - 超出范围的值将被归类到最近的边界切片


📊 时序对齐信息:
  - pred_next: 1 (预测未来第1根K线)
  - 原始数据长度: 1400
  - 对齐后数据长度: 1399
  - 丢弃的尾部数据: 1 条

已生成: slice_0.9900_1.0000_ratio_0.0036.jpg (切片区间: [0.9900, 1.0000] (做多), 样本占比: 0.36%)
已生成: slice_0.9800_0.9900_ratio_0.0043.jpg (切片区间: [0.9800, 0.9900] (做多), 样本占比: 0.43%)
已生成: slice_0.9700_0.9800_ratio_0.0029.jpg (切片区间: [0.9700, 0.9800] (做多), 样本占比: 0.29%)
已生成: slice_0.9600_0.9700_ratio_0.0029.jpg (切片区间: [0.9600, 0.9700] (做多), 样本占比: 0

In [ ]:
from src.features.simple_feature_calculator import SimpleFeatureCalculator
from research.model_pick.candle_fetch import FusionCandles
from strategies.BinanceBtcDeapV1Voting.models.config import SSMContainer
from jesse.helpers import date_to_timestamp

from pathlib import Path
import lightgbm as lgb
import pandas as pd
import numpy as np

T_START = "2025-01-01"
T_END = "2025-10-25"

path_features = Path(".").parent / "strategies" / f"{STRATEGY_NAME}" / "models" / "feature_info.json"
with open(path_features) as f:
    feature_info: dict[str, list[str]] = json.load(f)
FEAT_FRACDIFF: list[str] = feature_info["fracdiff"]
ALL_RAW_FEAT = []
ALL_RAW_FEAT.extend(FEAT_FRACDIFF)
ALL_RAW_FEAT.extend(feature_info[MODEL_NAME])
ALL_RAW_FEAT = set(ALL_RAW_FEAT)
ALL_RAW_FEAT = sorted(
    [
        i
        for i in ALL_RAW_FEAT
        if not i.startswith("deep_ssm") and not i.startswith("lg_ssm")
    ]
)

candle_container = FusionCandles(
    exchange="Binance Perpetual Futures", symbol="BTC-USDT", timeframe="1m"
)
candles = candle_container.get_candles(T_START, T_END)

feat_calc = SimpleFeatureCalculator()
feat_calc.load(candles, sequential=True)
print("getting raw features...")
df_raw_feats = pd.DataFrame.from_dict(feat_calc.get(ALL_RAW_FEAT))
df_raw_feats.set_index(candles[:, 0].astype(int), inplace=True)
print(f"{df_raw_feats.shape = }")

model_deep_ssm = SSMContainer(model_type="deep_ssm")
model_lg_ssm = SSMContainer(model_type="lg_ssm")

df_deep_ssm = model_deep_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_lg_ssm = model_lg_ssm.transform(df_raw_feats[FEAT_FRACDIFF])
df_feat_full = pd.concat([df_deep_ssm, df_lg_ssm, df_raw_feats], axis=1)

test_features = df_feat_full[
    (df_feat_full.index.to_numpy() >= date_to_timestamp(T_START)) &
    (df_feat_full.index.to_numpy() < date_to_timestamp(T_END))
]

In [ ]:
with open(f"strategies/{STRATEGY_NAME}/models/feature_info.json", "r") as f:
    feat_info = json.load(f)
df_selected_feats = test_features[feat_info[f"{MODEL_NAME}"]]

timestamps = candles[:, 0].astype(np.int64)
mask = (timestamps >= test_features.index[0]) & (timestamps <= test_features.index[-1])

close_price = candles[:, 2][mask]
datetimes = pd.to_datetime(test_features.index, unit="ms")
assert len(datetimes) == len(close_price)

model = lgb.Booster(model_file=f"strategies/{STRATEGY_NAME}/models/model_{MODEL_NAME}.txt")
pred = model.predict(df_selected_feats)
assert len(pred) == len(datetimes)

In [ ]:
from research.plot import analyze_confidence_slices

slice_res = analyze_confidence_slices(
    datetimes,
    pred,
    close_price,
    PRED_NEXT,
    output_dir=f"./temp/{MODEL_TYPE}_L{LAG}_N{PRED_NEXT}",
    granularity=0.02,
    lower_bound=LOWER_BOUND,
    upper_bound=UPPER_BOUND,
    threshold=THRESHOLD,
)

In [ ]:
from strategies.BinanceBtcDeapV1Voting.models.config import LGBMContainer

mc = LGBMContainer(
    model_type=MODEL_TYPE,
    lag=LAG,
    pred_next=PRED_NEXT,
    threshold=THRESHOLD,
)
print(mc.MODEL_NAME)

In [ ]:
# 根据图片设定合适的filters

GIVEUP_FILTERS = [
    (0, 0.02),
    (0.12, 0.14),
    (0.24, 0.26),
    (0.32, 0.38),
    (0.4, 0.42),
    (0.46, 0.48),
    (0.5, 0.52),
]

REVERSE_FILTERS = [
    (0.04, 0.08),
    (0.16, 0.18),
    (0.2, 0.24),
    (0.28, 0.3),
    (0.48, 0.5),
    (0.52, 0.54),
]

for f in GIVEUP_FILTERS:
    mc.add_giveup_filter(f[0], f[1])

for f in REVERSE_FILTERS:
    mc.add_reverse_filter(f[0], f[1])

In [ ]:
mc.save_filters()